In [1]:
pip install joblib

In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

pygame 2.3.0 (SDL 2.24.2, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'E:\FYP\models\model.h5')

In [ ]:
mixer.init()
sound = mixer.Sound(r'E:\FYP\Driver-Drowsiness-Detection-using-Deep-Learning-main\alarm.wav')
cap = cv2.VideoCapture(0)
blink_count = 0
blink_threshold = 3  # Adjust this threshold for blink detection
alarm_on = False

while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=1)

    for (ex, ey, ew, eh) in eyes:
        # Extract the region of the eye
        eye_region = frame[ey:ey + eh, ex:ex + ew]

        # Preprocessing steps
        eye = cv2.resize(eye_region, (80, 80))
        eye = eye / 255
        eye = eye.reshape(80, 80, 3)
        eye = np.expand_dims(eye, axis=0)

        # Make predictions using your model for this eye
        prediction = model.predict(eye)

        # Check if the eye is closed
        if prediction[0][0] > 0.30:  # Adjust the threshold for closed eyes
            blink_count += 1

        # Check if the eye is open
        if prediction[0][1] > 0.90:  # Adjust the threshold for open eyes
            blink_count = 0

        # Draw a bounding box around each detected eye
        cv2.rectangle(frame, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

        # If blink count exceeds the threshold, trigger an alarm
        if blink_count >= blink_threshold and not alarm_on:
            try:
                sound.play()
                alarm_on = True
            except:
                pass

        # If blink count returns to zero, stop the alarm
        if blink_count == 0 and alarm_on:
            sound.stop()
            alarm_on = False

    cv2.imshow('frame', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):  # Reduce the delay
        break

cap.release()
cv2.destroyAllWindows()
